## DATA

In [33]:
import random
import pandas as pd

obj = {}
for i in range(1, 21):
  obj['l' + str(i)] = [random.randrange(1, 100000, 1) for i in range(100000)]

data = pd.DataFrame(obj, columns = obj.keys())

target = [random.randrange(1, 100000, 1) for i in range(99000)]
nans = [None] * 1000
target_col = target + nans

data['target'] = target_col
data

,l1,l2,l3,l4,l5,l6,l7,l8,l9,l10,l11,l12,l13,l14,l15,l16,l17,l18,l19,l20,target
0,72141,34314,56316,44146,67407,91964,27753,35279,5019,15256,12434,3619,96553,27342,83835,17718,82183,34926,2318,92268,30278.0
1,11293,18134,40451,78112,46282,90249,34714,46317,22362,98829,57314,13424,18178,81190,73462,15386,84297,53485,1465,50309,51821.0
2,70739,59979,42067,23345,55755,61288,39674,55412,81487,1092,56128,94810,24782,73297,78811,5313,94741,17509,2375,70164,33756.0
3,82718,2545,58614,97831,47095,60029,31627,6388,22761,80036,68047,75271,23525,53823,89936,31460,80729,20719,9837,76924,98830.0
4,8233,12543,99173,50741,88974,45964,87581,6533,77727,94309,56855,81566,80930,79889,47874,49934,59415,95309,18236,39717,88341.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,3544,17533,33253,19994,91295,14950,89829,90871,694,80087,36090,95079,75144,65582,59889,73278,92249,59677,38034,19238,NaN
99996,48069,12397,76,89655,52940,39091,60638,793,72781,60171,54437,66669,17266,30527,42673,69511,49015,19635,97133,74541,NaN
99997,54909,84311,62511,95215,39059,33821,87104,85068,56354,10699,65572,34987,81421,89132,57441,46674,952,10724,29948,63400,NaN
99998,5266,38619,85401,44843,11606,81897,9231,98551,15009,87369,89257,22292,29997,34765,71275,47368,47214,3433,43276,80066,NaN


## PREPARATION

In [34]:
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np

yy = 'target'
data2 = data.dropna()
list_cor = []

for i in range(data2.shape[1]):
  a = round(np.corrcoef(pd.to_numeric(pd.Series(data2.iloc[:,i])), data2[yy])[0][1] , 2)
  list_cor.append(a)

list_cor = pd.Series(list_cor)
list_cor.index = data2.columns

y = list_cor.sort_values(ascending = False).index[0]
x = list_cor.sort_values(ascending = False).index[1]

y, x

('target', 'l20')

In [35]:
from sklearn.linear_model import LinearRegression

model = LinearRegression().fit(np.array(data2[x]).reshape((-1, 1)), np.array(data2[y]).reshape((-1, 1)))

data[y+'_ii'] = np.nan

for i in range(data.shape[0]):
  if data[y][i] is None:
    a = model.predict(np.array(data[x][i]).reshape((-1, 1)))[0][0]
    data[y + '_ii'][i] = a
  else:
    data[y + '_ii'][i] = data[x][i]

data.tail()

,l1,l2,l3,l4,l5,l6,l7,l8,l9,l10,l11,l12,l13,l14,l15,l16,l17,l18,l19,l20,target,target_ii
99995,3544,17533,33253,19994,91295,14950,89829,90871,694,80087,36090,95079,75144,65582,59889,73278,92249,59677,38034,19238,NaN,19238.0
99996,48069,12397,76,89655,52940,39091,60638,793,72781,60171,54437,66669,17266,30527,42673,69511,49015,19635,97133,74541,NaN,74541.0
99997,54909,84311,62511,95215,39059,33821,87104,85068,56354,10699,65572,34987,81421,89132,57441,46674,952,10724,29948,63400,NaN,63400.0
99998,5266,38619,85401,44843,11606,81897,9231,98551,15009,87369,89257,22292,29997,34765,71275,47368,47214,3433,43276,80066,NaN,80066.0
99999,98927,52734,1354,47244,83730,53324,48116,53806,78521,23224,59716,95352,86797,6037,60213,23350,64710,35388,24378,40968,NaN,40968.0


In [36]:
data.drop(y, axis = 1, inplace = True)
data.rename(columns = {y+'_ii' : y}, inplace = True)

data.columns

Index(['l1', 'l2', 'l3', 'l4', 'l5', 'l6', 'l7', 'l8', 'l9', 'l10', 'l11',
       'l12', 'l13', 'l14', 'l15', 'l16', 'l17', 'l18', 'l19', 'l20',
       'target'],
      dtype='object')

## FUNCTION

All non-quantitative variables must be deleted or transformed before running this function. If not will raise an error.

In [37]:
def na_cuant_replace(data, col_to_pred):

  from sklearn.linear_model import LinearRegression
  import pandas as pd
  import numpy as np
  import warnings
  warnings.filterwarnings('ignore')

  col_to_pred = col_to_pred
  data2 = data.dropna()
  list_cor = []

  for i in range(data2.shape[1]):
    a = round(np.corrcoef(pd.to_numeric(pd.Series(data2.iloc[:,i])), data2[col_to_pred])[0][1] , 2)
    list_cor.append(a)

  list_cor = pd.Series(list_cor)
  list_cor.index = data2.columns

  y = list_cor.sort_values(ascending = False).index[0]
  x = list_cor.sort_values(ascending = False).index[1]

  model = LinearRegression().fit(np.array(data2[x]).reshape((-1, 1)), np.array(data2[y]).reshape((-1, 1)))

  data[y + '_ii'] = np.nan

  for i in range(data.shape[0]):
    if data[y][i] is None:
      a = model.predict(np.array(data[x][i]).reshape((-1, 1)))[0][0]
      data[y + '_ii'][i] = a
    else:
      data[y + '_ii'][i] = data[x][i]

  data.drop(y, axis = 1, inplace = True)
  data.rename(columns = { y + '_ii' : y}, inplace = True)
  return data

In [38]:
na_cuant_replace(data = data, col_to_pred='target')

,l1,l2,l3,l4,l5,l6,l7,l8,l9,l10,l11,l12,l13,l14,l15,l16,l17,l18,l19,l20,target
0,72141,34314,56316,44146,67407,91964,27753,35279,5019,15256,12434,3619,96553,27342,83835,17718,82183,34926,2318,92268,92268.0
1,11293,18134,40451,78112,46282,90249,34714,46317,22362,98829,57314,13424,18178,81190,73462,15386,84297,53485,1465,50309,50309.0
2,70739,59979,42067,23345,55755,61288,39674,55412,81487,1092,56128,94810,24782,73297,78811,5313,94741,17509,2375,70164,70164.0
3,82718,2545,58614,97831,47095,60029,31627,6388,22761,80036,68047,75271,23525,53823,89936,31460,80729,20719,9837,76924,76924.0
4,8233,12543,99173,50741,88974,45964,87581,6533,77727,94309,56855,81566,80930,79889,47874,49934,59415,95309,18236,39717,39717.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,3544,17533,33253,19994,91295,14950,89829,90871,694,80087,36090,95079,75144,65582,59889,73278,92249,59677,38034,19238,19238.0
99996,48069,12397,76,89655,52940,39091,60638,793,72781,60171,54437,66669,17266,30527,42673,69511,49015,19635,97133,74541,74541.0
99997,54909,84311,62511,95215,39059,33821,87104,85068,56354,10699,65572,34987,81421,89132,57441,46674,952,10724,29948,63400,63400.0
99998,5266,38619,85401,44843,11606,81897,9231,98551,15009,87369,89257,22292,29997,34765,71275,47368,47214,3433,43276,80066,80066.0
